# Melhorias em implementação:

### Parte da classe atual ultrapassada:

In [222]:
from sgp4.api import SGP4_ERRORS

from astropy.coordinates import TEME, CartesianRepresentation  # CartesianDifferential
from astropy import units as u
from astropy.coordinates import ITRS

import pymap3d as pm

import pandas as pd
from sgp4 import omm
from sgp4.api import Satrec
from astropy.time import Time
from astropy.time import TimeDelta
import time
import numpy as np

class PropagInit:
    def __init__(self, satellite, lc, sample_time=1):
        self.satellite = satellite
        self.lc = lc
        self.sample_time = sample_time

        self.tempo = []
        self.traj = []
        self.dist = []
        self.hdmin = []
        self.dmin = []
        self.hrmin = []


    def orbit_propag(self, h0, n):
        time_array = np.linspace(h0,h0 + TimeDelta((n -1)* u.s),n)
        error_code, teme_p, teme_v = self.satellite.sgp4_array(time_array.jd1, time_array.jd2)
        teme_p = np.array(teme_p)
        x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
        teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
        teme = TEME(teme_p, obstime=time_array)  
        itrsp = teme.transform_to(ITRS(obstime=time_array))
        location = itrsp.earth_location
        enu_p = pm.ecef2enu(1000 * location.x.value,
                            1000 * location.y.value,
                            1000 * location.z.value, self.lc.lat,
                                    self.lc.lon, self.lc.height)

        enu_p = np.transpose(enu_p)
        enu_d = np.linalg.norm(enu_p,axis=1)
        min_index = np.argmin(enu_d)
        d_min = enu_d[min_index]
        posit_min = enu_p[min_index]
  
        self.tempo.append(time_array)
        self.traj.append(enu_p)
        self.dist.append(enu_d)
        self.hdmin.append(time_array[min_index])
        self.dmin.append(d_min)
        self.hrmin.append(min_index)
        return self

    def orbitpropag(self, t_inic, n):
        tvar = t_inic

        enu_dv = []
        tv = []
        enu_pv = []

        tsample = self.sample_time

        hr = 0

        flagmin = True
        hdmin = 0
        dmin = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0] ** 2 + enu_p[1] ** 2 + enu_p[2] ** 2) ** 0.5
            enu_dv.append(enu_d)
            enu_pv.append(enu_p)
            tv.append(tvar)

            if (enu_dv[len(enu_dv) - 2] < enu_dv[len(enu_dv) - 1]) and flagmin:
                print('NORAD: {} dmin= {:.3f}'.format(self.satellite.satnum, enu_dv[len(enu_dv) - 2]))
                dmin = enu_dv[len(enu_dv) - 2]
                hdmin = tvar - tsample / (60 * 60 * 24)
                hrmin = hr - 1
                flagmin = False

            hr += 1
            tvar += tsample / (60 * 60 * 24)
            if hr == n:
                self.tempo.append(tv)
                self.traj.append(enu_pv)
                self.dist.append(enu_dv)
                self.hdmin.append(hdmin)
                self.dmin.append(dmin)
                self.hrmin.append(hrmin)
        return self

    def searchh0(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            if enu_d > distmin:
                tsample = round(1+abs((enu_d - distmin)/10000))
                print("sample = {}".format(tsample))
                if not flagmin:
                    flagmin = True
                    self.tempo.append(tv)
                    self.traj.append(enu_pv)
                    self.dist.append(enu_dv)
                    self.hdmin.append(hdmin)
                    self.dmin.append(dmin)
                    self.hrmin.append(hrmin)
                    tv = []
                    enu_pv = []
                    enu_dv = []
                hr = 0
                hrmin = 0
            else:
                tsample = self.sample_time
                tv.append(tvar)
                enu_dv.append(enu_d)
                enu_pv.append(enu_p)
                # print('{} {:.3f}'.format(tvar.value, enu_d))
                if (enu_dv[len(enu_dv)-2] < enu_dv[len(enu_dv)-1]) and flagmin:
                    dmin = enu_dv[len(enu_dv) - 2]
                    if dmin < distmin2:
                        hdmin = tvar - TimeDelta(tsample * u.s)
                        hrmin = hr - 1
                        flagmin = False
                        ntraj += 1
                        print('NORAD: {}, H0: {}, dmin= {:.3f}'.format(self.satellite.satnum, tv[0].value, dmin))
                    else:
                        tsample = 60*5

            hr += 1
            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

    def searchh01(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            # if enu_d < distmin:
            #     print('ok')
            if (enu_d - distmin)>0:
                tsample = round(5+abs(1*(enu_d - distmin)/8000))
            else:
                tsample = round(5+abs(2*(enu_d - distmin)/8000))            

            print("sample = {}, dist = {}".format(tsample,enu_d ))

            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

    def searchh02(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        sign = 1

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            # if enu_d < distmin:
            #     print('ok')
            if (enu_d - distmin)>0:
                tsample = sign * round(5+abs(1*(enu_d - distmin)/8000))
            else:
                tvar += TimeDelta(60*10 * u.s)
                if sign > 0:  sign = -1
                else: sign = 1                              

            print("sample = {}, dist = {}".format(tsample,enu_d ))

            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

    def pos_calc(self,tvar):
        error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
        teme_p = CartesianRepresentation(teme_p * u.km)
        teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
        itrsp = teme.transform_to(ITRS(obstime=tvar))
        location = itrsp.earth_location
        # location.geodetic
        enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                            self.lc.lon, self.lc.height)
        enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5
        return enu_p, enu_d

class LocalFrame:
    def __init__(self, lat, lon, height):
        self.lat = lat
        self.lon = lon
        self.height = height

## Encontrar H0 atual:

In [ ]:


orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

satellite = Satrec()
omm.initialize(satellite, orbital_elem[20])

posit = PropagInit(satellite,lc)


t_inic=Time('2022-08-10T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-12T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

ini = time.time()
posit.searchh02(t_inic, t_final, 1100*1000,  1000*1000,  10000)
fim = time.time()
print("Tempo: ", fim - ini)

In [ ]:
posit.tempo

In [13]:
orbital_elem[20]['TLE_LINE1']

'1 41335U 16011A   22222.69171522  .00000046  00000-0  37168-4 0  9997'

In [15]:
orbital_elem[20]['TLE_LINE2']

'2 41335  98.6197 289.1969 0001170  98.9531 261.1782 14.26739266337507'

## Melhoria Encontrar H0 rápido:

In [93]:
from skyfield.api import load, wgs84

tles1 = load.tle_file('conftle.txt')

#tles2 = load.tle_file()

sat = tles1[0]
print(sat)

ini = time.time()

ts = load.timescale()

bluffton = wgs84.latlon(-5.92256, -35.1615)
t0 = ts.utc(2022, 8, 10)
t1 = ts.utc(2022, 8, 12)
t, events = sat.find_events(bluffton, t0, t1, altitude_degrees=30.0)

fim = time.time()
print("Tempo: ", fim - ini)

ini = time.time()
for ti, event in zip(t, events):
    #name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    #print(ti.utc_strftime('%Y-%m-%dT%H:%M:%S'), name)

    tem = Time(ti.utc_strftime('%Y-%m-%dT%H:%M:%S'), format='isot')
    enup, enud = posit.pos_calc(tem) ## Lento
    #print(enup)
fim = time.time()
print("Tempo teme to enu: ", fim - ini)

catalog #41335 epoch 2022-08-10 16:36:04 UTC
Tempo:  0.0070002079010009766
Tempo teme to enu:  0.016000032424926758


## Melhoria Código rápido de calculo trajetória:

In [95]:


tem = Time(t.utc_strftime('%Y-%m-%dT%H:%M:%S'), format='isot')

ini = time.time()

error_code, teme_p, teme_v = satellite.sgp4_array(tem.jd1, tem.jd2)

teme_p = np.array(teme_p)
x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]

teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
teme = TEME(teme_p, obstime=tem)  # ver diferença entre UT1 e UTC
itrsp = teme.transform_to(ITRS(obstime=tem))
location = itrsp.earth_location
enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                            lc.lon, lc.height)

fim = time.time()
print("Tempo: ", fim - ini)

enu_p

Tempo:  0.003001689910888672


(array([   25466.06337329,  -234535.48500898,  -485454.81320164,
         -878175.99713957, -1024634.30626479, -1164614.57977876,
          780143.46398059,   540860.39993516,   290103.4642783 ]),
 array([ 1216598.09923688,    51033.27412469, -1116505.87515824,
         -843261.56259651,  -250690.24273257,   343486.73060408,
          927915.67102392,  -127756.80189316, -1188335.8860551 ]),
 array([700461.93302992, 802338.92532322, 704343.79299177, 704159.42803241,
        729241.30021246, 702741.30093862, 701815.80867844, 785162.04318144,
        703506.06784671]))

### Funcionando

In [262]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()

for index, row in df_conf.iterrows():
    satellite = Satrec()
    omm.initialize(satellite, orbital_elem[index])
    h0 = Time(row['H0'], format='isot')
    time_array = np.linspace(h0,h0 + TimeDelta((row['N_PT'] -1)* u.s),row['N_PT'])

    error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
    teme_p = np.array(teme_p)
    x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]

    teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
    teme = TEME(teme_p, obstime=time_array)  
    itrsp = teme.transform_to(ITRS(obstime=time_array))
    location = itrsp.earth_location
    enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                                lc.lon, lc.height)

    enu_p = np.transpose(enu_p)
    #enu_p = np.reshape(np.asarray(enu_p), (row['N_PT'], 3)) 
    enu_d = np.linalg.norm(enu_p,axis=1)
    min_index = np.argmin(enu_d)
    d_min = enu_d[min_index]
    posit_min = enu_p[min_index]
    
    break

fim = time.time()
print("Tempo: ", fim - ini)

# print(enu_d)



Tempo:  0.0039975643157958984


In [263]:
np.shape(enu_p)

(233, 3)

In [264]:
enu_p[0]

array([-623507.73004444, -534606.09316655,  729029.18855509])

In [265]:
np.linalg.norm(enu_p,axis=1)

array([1098202.6780341 , 1093516.98920657, 1088851.52577009,
       1084206.55366501, 1079582.34221243, 1074979.16413473,
       1070397.29557609, 1065837.01612143, 1061298.60881421,
       1056782.360173  , 1052288.56020608, 1047817.50242535,
       1043369.48385789, 1038944.80505603, 1034543.77010669,
       1030166.68663626, 1025813.86581662, 1021485.62236743,
       1017182.27455522, 1012904.14419359, 1008651.55663674,
       1004424.8407744 , 1000224.32902084,  996050.35730444,
        991903.26505141,  987783.39516848,  983691.09402198,
        979626.71141369,  975590.60055296,  971583.11802678,
        967604.62376432,  963655.48099958,  959736.05622872,
        955846.71916548,  951987.84268969,  948159.80279427,
        944362.97852646,  940597.75192455,  936864.50795125,
        933163.6344205 ,  929495.52192177,  925860.5637364 ,
        922259.15575212,  918691.69637043,  915158.58640865,
        911660.22899769,  908197.02947307,  904769.39526156,
        901377.73576099,

In [238]:
np.shape(enu_p)


(233, 3)

In [237]:
np.shape(enu_d)


(233,)

In [211]:
enu_d

13684736.927059043

In [151]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()

for index, row in df_conf.iterrows():
    satellite = Satrec()
    orbital_elem_row = next(x for x in orbital_elem if x["NORAD_CAT_ID"] == row['NORAD_CAT_ID'])
    omm.initialize(satellite, orbital_elem_row)
    propag = PropagInit(satellite, lc, 1) 
    pos = propag.orbitpropag(Time(row['H0'], format='isot'),row['N_PT'])
    
    break

fim = time.time()
print("Tempo: ", fim - ini)


NORAD: 2142 dmin= 776821.976
Tempo:  0.39698338508605957


In [232]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()
sdf = SummarizeDataFiles()
for index, row in df_conf.iterrows():
    satellite = Satrec()
    orbital_elem_row = next(x for x in orbital_elem if x["NORAD_CAT_ID"] == row['NORAD_CAT_ID'])
    omm.initialize(satellite, orbital_elem_row)
    propag = PropagInit(satellite, lc, 1) 
    pos = propag.orbit_propag(Time(row['H0'], format='isot'),row['N_PT'])
    sdf.save_trajectories(pos,orbital_elem_row)
    break

fim = time.time()
print("Tempo: ", fim - ini)

Tempo:  0.0039997100830078125


In [200]:
np.shape(pos.dist )


(1, 3)

In [231]:
class SummarizeDataFiles:
    def __init__(self):
        self.sel_orbital_elem = []
        self.sel_resume = { "H0":[], "DIST_H0":[],"H_DIST_MIN":[],"PT_DIST_MIN":[],
                    "DIST_MIN":[],"HF":[], "N_PT":[], "DIST_HF":[],"RCS":[] }

    def save_trajectories(self,pos,orbital_elem): 
        for i in range(0, len(pos.traj)):
            tempo = pos.tempo[i]
            posenu = pos.traj[i]
            distenu = pos.dist[i]
            
            ttxt = tempo[0].strftime('%Y_%m_%d-H0-%H_%M_%S')

            # Summarized data set of the trajectories obtained
            self.sel_orbital_elem.append(orbital_elem) 

            self.sel_resume["RCS"].append(orbital_elem['RCS_SIZE'])
            self.sel_resume["H0"].append(tempo[0].value)
            self.sel_resume["DIST_H0"].append(distenu[0])
            self.sel_resume["H_DIST_MIN"].append(pos.hdmin[i].value[11:])
            self.sel_resume["PT_DIST_MIN"].append(pos.hrmin[i])
            self.sel_resume["DIST_MIN"].append(pos.dmin[i])
            self.sel_resume["HF"].append(tempo[len(tempo) - 1].value[11:])
            self.sel_resume["N_PT"].append(len(distenu) - 1)
            self.sel_resume["DIST_HF"].append(distenu[len(distenu) - 1]) 

In [233]:
sdf.sel_resume

{'H0': ['2022-08-06T12:30:39.000'],
 'DIST_H0': [1070257.5070111575],
 'H_DIST_MIN': ['12:32:39.000'],
 'PT_DIST_MIN': [120],
 'DIST_MIN': [11266.620239357559],
 'HF': ['12:34:31.000'],
 'N_PT': [232],
 'DIST_HF': [1261516.725177206],
 'RCS': ['LARGE']}